In [1]:
from deap import base, creator, tools  # deap utilitieso
import random
import numpy as np  # numerical computation
import matplotlib.pyplot as plt   # plotting

In [2]:
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", np.ndarray, fitness=creator.FitnessMax)

In [3]:
IND_SIZE = 3  # chromosome length

toolbox = base.Toolbox()
toolbox.register("attribute", random.randint,0, 20)
toolbox.register("individual", tools.initRepeat, creator.Individual,
                 toolbox.attribute, n=IND_SIZE)
toolbox.register("population", tools.initRepeat, list,toolbox.individual)


In [28]:
def evaluate(ind):
    """ Returns the fitness of an individual.
    This is your objective function."""
    F=4*ind[0]+8*ind[1]+12*ind[2]-15*((6*ind[0]+3*ind[1]+12*ind[2])/120)**10
    lista=[]
    lista.append(F)
    return sum(lista),  #  must return a tuple

# Operators
toolbox.register("cross", tools.cxTwoPoint)  
toolbox.register("mutate", tools.mutUniformInt, low=0, up=20, indpb=1.0)
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("evaluate", evaluate)

In [46]:
def generationalGA():
    pop = toolbox.population(n=1000)  # Registered as a list
    CXPB = 0.5  # Crossover probability
    MUTPB = 0.2  # Mutation probability
    NGEN = 100 # Number of generations
    
    # Evaluate all population first
    fitnesses = map(toolbox.evaluate, pop)
    for ind, fit in zip(pop, fitnesses):
        ind.fitness.values = fit
    
    # Generate offspring
    for __ in range(NGEN):
        offspring = toolbox.select(pop, len(pop))
        offspring = list(map(toolbox.clone, offspring))  # Generate a deep copy
        
        # Apply crossover
        for c1, c2 in zip(offspring[::2], offspring[1::2]):
            if random.random() < CXPB:
                toolbox.cross(c1, c2)
                # Reset their fitness values
                del c1.fitness.values
                del c2.fitness.values

        # Mutate those selected
        for mutant in offspring:
            if random.random() < MUTPB:
                toolbox.mutate(mutant)
                # Reset fitness values
                del mutant.fitness.values

        # Evaluate non-evaluated individuals in offspring
        invalid_inds = [ind for ind in offspring if not ind.fitness.valid]
        fitnesses = map(toolbox.evaluate, invalid_inds)
        for ind, fit in zip(invalid_inds, fitnesses):
            ind.fitness.values = fit

        # Replace entire population by the offspring
        pop[:] = offspring
    
    return pop

In [47]:
results = generationalGA()
fitnesses = [i.fitness.getValues()[0] for i in results]

bestpos = fitnesses.index(max(fitnesses))
print("Individual: {0}\n\nFitness: {1}".format(
        results[bestpos], results[bestpos].fitness.getValues()[0]))
motores=6*results[bestpos][0]+3*results[bestpos][1]+12*results[bestpos][2]
print(motores)


Individual: [ 0 20  5]

Fitness: 205.0
120
